# Machine Learning
## Assignment - 2

Student Name: Rohit Garla

Email ID: rg3365@nyu.edu

In [205]:
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import *
import operator
from collections import OrderedDict
import pandas as pd
import matplotlib.pyplot as plt
from itertools import islice
%matplotlib inline

In [227]:
train = pd.read_csv("reviewstrain.txt", delimiter="\t", header=None)
train = pd.DataFrame(train)
train = pd.DataFrame(train[0].astype(str).str.split(' ',1).tolist(), columns = ['label','reviews'])
train["label"] = train['label'].astype(int)
test = pd.read_csv("reviewstest.txt", delimiter="\t", header=None)
test = pd.DataFrame(test)
test = pd.DataFrame(test[0].astype(str).str.split(' ',1).tolist(), columns = ['label','reviews'])
test["label"] = test['label'].astype(int)
train.head()


for i in range(len(train)):
    train['reviews'][i]=train['reviews'][i].split(' ')
for i in range(len(test)):
    test['reviews'][i]=test['reviews'][i].split(' ')
#test.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [207]:
test.head()

,label,reviews
0,1,"funny , sexy , devastating and incurably roman..."
1,1,cool gadgets and creatures keep this fresh .
2,1,"fathers and sons , and the uneasy bonds betwee..."
3,0,after that it becomes long and tedious like a ...
4,0,what should have been a cutting hollywood sati...


In [214]:
for x in train['reviews'][1]:
    print (x)

a
real
winner
--
smart
,
funny
,
subtle
,
and
resonant
.


In [213]:
train['reviews'][1]

['a',
 'real',
 'winner',
 '--',
 'smart',
 ',',
 'funny',
 ',',
 'subtle',
 ',',
 'and',
 'resonant',
 '.']

In [180]:
def inter(c1,c2):
    unique=set(c2).intersection(c1)
    if len(unique)==0:
        return np.nan
    else:
        return (1/len(unique))      

In [181]:
def neighbours(k,testSample):
    sorted_distances_dataframe = testSample.sort_values('Distance')
    neighbour = sorted_distances_dataframe.head(k)
    last_distance = sorted_distances_dataframe[(k-1):].Distance.values[0]
    sorted_distances_dataframe = sorted_distances_dataframe.drop(sorted_distances_dataframe.index[0:k])
    sorted_distances_dataframe = sorted_distances_dataframe[sorted_distances_dataframe['Distance']==last_distance]
    
    neighbour=neighbour.append(sorted_distances_dataframe)
    
    return neighbour
    

In [199]:
def predictionKnn(k,test,train):
    train_instance_Dataframe=train
    train_instance_Dataframe["Distance"]=0
    final_test=test
    final_test["Prediction"]=1
    distance=[]
    c0=0
    c1=0
    for x in test.index.values:
        for y in train.index.values:
            distance.append(inter(test['reviews'][x],train['reviews'][y]))
        #print(len(distance))
        train_instance_Dataframe["Distance"]=distance
        neighbour = neighbours(k,train_instance_Dataframe).reset_index()
        #return neighbour
        for i in range(len(neighbour)):
            if neighbour['label'][i]==0 or neighbour['label'][i]=='0':
                c0=c0+1
            else:
                c1=c1+1
        if(c0>c1):
            final_test['Prediction'][x]=0
        c0=0
        c1=0
        distance=[]
    return final_test

In [183]:
def metric_variables(k,test,train):
    true_positive=0
    true_negative=0
    false_positive=0
    false_negative=0
    final_test=predictionKnn(k,test,train)
    for i in range(len(final_test)):
        if final_test['label'][i]==1:
            if final_test['Prediction'][i]==1:
                true_positive=true_positive+1
            else:
                false_negative=false_negative+1
        else:
            if final_test['Prediction'][i]==0:
                true_negative=true_negative+1
            else:
                false_positive=false_positive+1
    print("CONFUSION MATRIX")
    print("True Positive = "+str(true_positive))
    print("True Negative = "+str(true_negative))
    print("False Negative = "+str(false_negative))
    print("False Positive = "+str(false_positive))
    print('Accuracy = '+str((true_positive+true_negative)/(true_positive+true_negative+false_positive+false_negative)))
    print('True Positive Rate = '+ str(true_positive/(true_positive+false_negative)))
    print('False Positive Rate = '+str(false_positive/(false_positive+true_negative)))

In [184]:
metric_variables(1,test,train)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


CONFUSION MATRIX
True Positive = 209
True Negative = 93
False Negative = 64
False Positive = 134
Accuracy = 0.604
True Positive Rate = 0.7655677655677655
False Positive Rate = 0.5903083700440529


In [185]:
metric_variables(5,test,train)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


CONFUSION MATRIX
True Positive = 212
True Negative = 91
False Negative = 61
False Positive = 136
Accuracy = 0.606
True Positive Rate = 0.7765567765567766
False Positive Rate = 0.5991189427312775


In [186]:
def zeroR():
    true_positive=0
    true_negative=0
    false_positive=0
    false_negative=0
    c0=0
    c1=0
    c=0
    for i in range(len(train)):
        if train['label'][i]==0:
            c0=c0+1
        else:
            c1=c1+1
        if c1>c0:
            c=c1
        else:
            c=c0
    final_test=test
    final_test['Prediction']=c
    for i in range(len(final_test)):
        if final_test['label'][i]==1:
            if final_test['Prediction'][i]==1:
                true_positive=true_positive+1
            else:
                false_negative=false_negative+1
        else:
            if final_test['Prediction'][i]==0:
                true_negative=true_negative+1
            else:
                false_positive=false_positive+1
    print("CONFUSION MATRIX")
    print("True Positive = "+str(true_positive))
    print("True Negative = "+str(true_negative))
    print("False Negative = "+str(false_negative))
    print("False Positive = "+str(false_positive))

In [187]:
zeroR()

CONFUSION MATRIX
True Positive = 0
True Negative = 0
False Negative = 273
False Positive = 227


In [188]:
def crossValidation(klist,test,train):
    df = pd.DataFrame(columns=['label','reviews','Prediction'])
    kaccu={}
    for k in klist:
        for i in range(5):
            #print(i)
            test_data = train[(i*300):(i*300)+300]
            train_data = train.drop(train.index[(i*300):(i*300)+300])
            #train_data = train_data.reset_index(drop=True)
            df1 = predictionKnn(k,test_data,train_data)
            df=df.append(df1)
        correct_pred =0
        total_pred=0
        for row in df.itertuples():
            total_pred = total_pred + 1
            if int(row.label) == int(row.Prediction):
                correct_pred =correct_pred + 1
        print("For K = ", k)
    #print('Correction Predictions = ', correct_pred)
    #print('Total Predictions = ', total_pred)
        accuracy = (correct_pred/total_pred)*100
        print('Accuracy = ', accuracy)
        print("----------------------------------------------------------------------")
        kaccu.update({k : accuracy})

In [189]:
crossValidation([3,7,99],test,train)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


For K =  3
Accuracy =  66.06666666666666
----------------------------------------------------------------------
For K =  7
Accuracy =  65.96666666666667
----------------------------------------------------------------------
For K =  99
Accuracy =  64.37777777777778
----------------------------------------------------------------------


In [190]:
metric_variables(3,test,train)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


CONFUSION MATRIX
True Positive = 212
True Negative = 83
False Negative = 61
False Positive = 144
Accuracy = 0.59
True Positive Rate = 0.7765567765567766
False Positive Rate = 0.6343612334801763


In [223]:
def stemming_elstopwords(dataframe):
    # Remove special characters, numbers, punctuations
    dataframe['reviews'] = dataframe['reviews'].str.replace("[^a-zA-Z#]", " ")
    
    # Removing Short Words
    dataframe['reviews'] = dataframe['reviews'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
    
    # Stemming
    stemmer = PorterStemmer()
    dataframe = dataframe.apply(lambda x: [stemmer.stem(i) for i in x]) 
    
    # Removing of Stop Words
    stop = stopwords.words('english')
    dataframe['reviews'] = dataframe['reviews'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    
    return dataframe

In [ ]:
train1 = pd.read_csv("reviewstrain.txt", delimiter="\t", header=None)
train1 = pd.DataFrame(train1)
train1 = pd.DataFrame(train1[0].astype(str).str.split(' ',1).tolist(), columns = ['label','reviews'])
trains = stemming_elstopwords(train1)
for i in range(len(train1)):
    train1['reviews'][i]=train1['reviews'][i].split(' ')
train1["label"] = trains['label'].astype(int)
test1 = pd.read_csv("reviewstest.txt", delimiter="\t", header=None)
test1 = pd.DataFrame(test1)
test1 = pd.DataFrame(test1[0].astype(str).str.split(' ',1).tolist(), columns = ['label','reviews'])
tests = stemming_elstopwords(test1)
test1["label"] = tests['label'].astype(int)
train1.head()
for i in range(len(test1)):
    test1['reviews'][i]=test1['reviews'][i].split(' ')

In [ ]:
import sys
if sys.version_info[0] >= 2:
    raw_input = input
user_input = raw_input("1: Default Distance\n2: Refined Dataframe Distance\n")
user_input

In [230]:
#for k=1
if (user_input==1):
    metric_variables(1,test,train)
else:
    metric_variables(1,test1,train1)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


CONFUSION MATRIX
True Positive = 211
True Negative = 129
False Negative = 62
False Positive = 98
Accuracy = 0.68
True Positive Rate = 0.7728937728937729
False Positive Rate = 0.43171806167400884


In [232]:
#for k=5
metric_variables(5,test1,train1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


CONFUSION MATRIX
True Positive = 211
True Negative = 136
False Negative = 62
False Positive = 91
Accuracy = 0.694
True Positive Rate = 0.7728937728937729
False Positive Rate = 0.4008810572687225
